In [1]:
!git clone https://github.com/Team-M1/badwords-classifier-train -b BM # 자기 브랜치 이름으로 변경
%cd badwords-classifier-train
!pip install -r requirements.txt
!pip install sentencepiece

Cloning into 'badwords-classifier-train'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 113 (delta 59), reused 71 (delta 17), pack-reused 0
Receiving objects: 100% (113/113), 2.73 MiB | 11.05 MiB/s, done.
Resolving deltas: 100% (59/59), done.
/content/badwords-classifier-train
     |████████████████████████████████| 264 kB 970 kB/s 
     |████████████████████████████████| 272 kB 43.5 MB/s 
     |████████████████████████████████| 2.8 MB 28.9 MB/s 
     |████████████████████████████████| 918 kB 42.0 MB/s 
     |████████████████████████████████| 302 kB 22.4 MB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 243 kB 54.7 MB/s 
     |████████████████████████████████| 119 kB 52.6 MB/s 
     |████████████████████████████████| 895 kB 49.7 MB/s 
     |████████████████████████████████| 636 kB 50.5 MB/s 
     |████████████████████████████████

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path_to_drive = "/content/drive/MyDrive/Colab Notebooks"

Mounted at /content/drive


# 시드 통일하기

In [3]:
import torch
from transformers import set_seed


set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# 모델 불러오기

In [4]:
# 만약 ForSequenceClassification이 붙은 모델을 사용할 경우
model_config = {
    "num_labels": 3,
    "id2label": {0: 0, 1: 1, 2: 2},
    "label2id": {0: 0, 1: 1, 2: 2}
}

In [7]:
from transformers import DistilBertForSequenceClassification
from tokenization_kobert import KoBertTokenizer

model = DistilBertForSequenceClassification.from_pretrained("monologg/distilkobert", **model_config)
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert", model_max_length=512)

# model.save_pretrained("./model")
tokenizer.tokenize("안녕하세요. 오늘 날씨는 비가 계속 내리내요.")

Downloading:   0%|          | 0.00/441 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/distilkobert were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at monologg/distilkobert and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


['▁안', '녕', '하세요', '.', '▁오늘', '▁날씨', '는', '▁비가', '▁계속', '▁내리', '내', '요', '.']

# 하이퍼 파라미터 설정

In [ ]:
num_classes = 3

# 원하는 대로 고쳐서 사용
batch_size = 64
lr = 5e-5  # 0.00005
epochs = 15

In [ ]:
# 옵티마이저와 스케줄러
# 원하는 대로 고쳐서 사용

# from torch.optim import AdamW
# from transformers import AdamW, get_linear_schedule_with_warmup


# LENGTH_OF_TRAIN_DATA = 40242
# num_training_steps = ((LENGTH_OF_TRAIN_DATA - 1) // batch_size + 1) * epochs
# optimizer = AdamW(model.parameters(), lr=lr)
# scheduler = get_linear_schedule_with_warmup(optimizer, int(num_training_steps * 0.1), num_training_steps)

# 만약 이 스케줄러를 사용할 경우,
# optimizer.step() 바로 다음에
# scheduler.step()을 호출해야 함

# f1 score 계산하기

In [ ]:
# 1. torchmetrics 사용
# requirements에 넣어놨으니 따로 설치할 필요 없음

# from torchmetrics import F1


# f1_score = F1(num_classes=num_classes)

# # 검증 단계에서 사용
# for inputs, labels in val_loader:
#     # 대충 코드
#     output = model(inputs)  # 대충 아웃풋
#     pred = torch.argmax(output, dim=1)
#     batch_f1 = f1_score(pred, labels)
#     print(batch_f1)

# f1 = f1_score.compute()
# print(f1)

In [ ]:
# 2. datasets 사용
# requirements에 넣어놨으니 따로 설치할 필요 없음
# 3번 사이킷런 사용법 쓰십시오 이건 좋지 않음

# from datasets import load_metric


# f1_score = load_metric("f1")

# for inputs, labels in val_loader:
#     # 대충 코드
#     output = model(inputs)
#     pred = torch.argmax(output, dim=1)
#     f1_score.add_batch(predictions=pred, references=labels)

# f1 = f1_score.compute()
# print(f1)

In [ ]:
# 3. sklearn 사용
# from sklearn.metrics import accuracy_score, f1_score


# def compute_metrics(p):
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)

#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     f1 = f1_score(y_true=labels, y_pred=pred)

#     return {"accuracy": accuracy, "f1": f1}

# **훈련**

In [5]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred, average="macro")

    return {"accuracy": accuracy, "f1": f1}

In [8]:
from data_loader import get_data_loaders


train_data, val_data, test_data = get_data_loaders(tokenizer, return_loader=False)
train_data = train_data.remove_columns("token_type_ids")
val_data = val_data.remove_columns("token_type_ids")
test_data = test_data.remove_columns("token_type_ids")

Using custom data configuration default-03a91ad376cd242b


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-03a91ad376cd242b/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/40242 [00:00<?, ?ex/s]

  0%|          | 0/4472 [00:00<?, ?ex/s]

  0%|          | 0/5000 [00:00<?, ?ex/s]

In [9]:
model_name = "DistilKoBERT"
save_path = f"{path_to_drive}/{model_name}"

In [10]:
from transformers import TrainingArguments, EarlyStoppingCallback

from trainer import ImbalancedSamplerTrainer

args = TrainingArguments(
    output_dir= save_path,
    overwrite_output_dir=True,
    evaluation_strategy="steps",
    num_train_epochs=15,
    logging_steps=1000,
    save_steps=1000,
    seed=42,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_ratio=0.1,
)

trainer = ImbalancedSamplerTrainer(
    model=model,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    # tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 40242
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 37740


Step,Training Loss,Validation Loss,Accuracy,F1
1000,0.950500,0.633172,0.754025,0.497500


***** Running Evaluation *****
  Num examples = 4472
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/DistilKoBERT/checkpoint-1000
Configuration saved in /content/drive/MyDrive/Colab Notebooks/DistilKoBERT/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/DistilKoBERT/checkpoint-1000/pytorch_model.bin


In [ ]:
trainer.save_model(output_dir=save_path)

In [ ]:
trainer.savestate()